In [ ]:
import torch
import torch.linalg as tla
import torch.autograd
import numpy as np
import matplotlib.pyplot as plt
import numml.sparse as sp
import numml.utils as utils
import scipy.linalg as sla

In [ ]:
# Create our favorite poisson operator

N = 7
A = sp.eye(N)*2. - sp.eye(N, k=1) - sp.eye(N, k=-1)
A.requires_grad = True
print(A.to_dense())

In [ ]:
LU = sp.splu(A)
LU.to_dense()

In [ ]:
L = LU.tril(k=-1) + sp.eye(N)
U = LU.triu()

print(torch.allclose((L@U).to_dense(), A.to_dense()))

In [ ]:
b = torch.zeros(N)
b[N//2] = 1.
b.requires_grad = True

x = sp.spsolve(A, b)
x

In [ ]:
plt.plot(b.detach().numpy())
plt.plot(x.detach().numpy())

In [ ]:
def solve(A):
    b = torch.ones(N)
    return sp.spsolve(A, b).sum() ** 2.

print(torch.round(utils.sp_fd(solve, A.detach()).to_dense()))
print(torch.round(sp.SparseCSRTensor((torch.autograd.grad(solve(A), A.data)[0], A.indices, A.indptr), A.shape).to_dense()))